In [3]:
import os
import json
import pandas as pd
import traceback

In [21]:
from langchain_openai import ChatOpenAI

In [9]:
KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
print(KEY)

In [6]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [29]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F81D45F770>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F81D8C42F0>, root_client=<openai.OpenAI object at 0x000001F81D45C980>, root_async_client=<openai.AsyncOpenAI object at 0x000001F81D8C4050>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [22]:

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import pypdf

In [26]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [24]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [25]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [35]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [30]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [31]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [32]:

review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [36]:

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [42]:
file_path=r"C:\Users\nutne\codes\MCQ\data.txt"

In [44]:
file_path

'C:\\Users\\nutne\\codes\\MCQ\\data.txt'

In [45]:

with open(file_path, 'r') as file:
    TEXT = file.read()

In [46]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probably approxima

In [66]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [67]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [68]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on explor

In [69]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1597
Prompt Tokens:880
Completion Tokens:717
Total Cost:0.0015155000000000001


In [70]:
quiz=response.get("quiz")

In [71]:
quiz=json.loads(quiz)

In [72]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [73]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: A field of study in artificial intelligence concerned with the development and study of statistical algorithms | b: A type of computer hardware | c: A programming language | d: A type of software application',
  'Correct': 'a'},
 {'MCQ': 'Which field does machine learning find application in?',
  'Choices': 'a: Biology | b: Natural language processing, computer vision, speech recognition | c: Chemistry | d: Physics',
  'Correct': 'b'},
 {'MCQ': 'What is the foundation of machine learning?',
  'Choices': 'a: Chemistry | b: Statistics and mathematical optimization | c: Biology | d: Physics',
  'Correct': 'b'},
 {'MCQ': 'What is data mining focused on?',
  'Choices': 'a: Exploratory data analysis via supervised learning | b: Exploratory data analysis via unsupervised learning | c: Predictive analytics | d: Natural language processing',
  'Correct': 'b'},
 {'MCQ': 'What framework provides a theoretical viewpoint for describing machine 

In [76]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What is machine learning?,a: A field of study in artificial intelligence...,a
1,Which field does machine learning find applica...,"a: Biology | b: Natural language processing, c...",b
2,What is the foundation of machine learning?,a: Chemistry | b: Statistics and mathematical ...,b
3,What is data mining focused on?,a: Exploratory data analysis via supervised le...,b
4,What framework provides a theoretical viewpoin...,a: Probably approximately correct learning | b...,a


In [78]:
pd.DataFrame(quiz_table_data).to_csv("machinelearning.csv", index=False)